# Quote Guessing Game

## Introduction

In this notebook, we scrape quotes from [quotes.toscrape.com](https://quotes.toscrape.com) and use them to create a guessing game where the player guesses the author of the quote. The game score is saved in a file in your Downloads folder after each session.

## Importing Libraries

In [1]:
import os
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

## Web Scraping

In [2]:
#Scrapping Quotes from random page of website
def scrappingQuoteDetails():
    pageNum = random.randint(1,10)
    url = f'https://quotes.toscrape.com'
    response = requests.get(f'{url}/page/{pageNum}/', headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"})
    data = response.content
    soup = BeautifulSoup(data, 'html.parser')
    
    randQuote = []
    quoteBy = []
    authorLink = []
    
    quotes = soup.findAll('div', class_='quote')
    for quote in quotes:
        randQuote.append(quote.find(class_='text').text.strip("“”"))
        quoteBy.append(quote.find(class_='author').text)
        authorLink.append(quote.find('a')['href'])

    return url, randQuote, quoteBy, authorLink
    
#Selecting random quote to display from the scraped quotes & scrapping author name and details
def quoteForDisplay():
    url, randQuote, quoteBy, authorLink = scrappingQuoteDetails()
    
    quoteToDisplay = random.randint(0,9)
    quote = randQuote[quoteToDisplay]
    auth = quoteBy[quoteToDisplay]
    
    authLink = f'{url}{authorLink[quoteToDisplay]}'
    authPageResponse = requests.get(authLink)
    authData = authPageResponse.content
    soup = BeautifulSoup(authData, 'html.parser')
    
    authBornDate = soup.find('span', class_='author-born-date').text
    authBirthPlace = soup.find('span', class_='author-born-location').text
    
    return quote, auth, authBornDate, authBirthPlace

## Game Flow & Dataframe

In [3]:
#Getting first letter of last name
def lastNameHint(auth):
    try:
        return (auth.split(' ')[2])[0]
    except:
        return (auth.split(' ')[1])[0]

#Getting time the game was played
def timeNow():
    now = datetime.now()
    return now.strftime("%d/%b/%y %I:%M %p")

#Declearing game won or lost
def wonOrlost(tries):
    if tries == 0:
        return 'Lost'
    else:
        return 'Won'

In [4]:
#Dictionary to store game info
table = {'Time': [], 'Won-Lost': [], 'Attempts Remaining':[]}

#While loop to run game as many times as player wants
wantToPlay = 'y'
while wantToPlay in ['y', 'Y']:
    
    quote, auth, authBornDate, authBirthPlace = quoteForDisplay()
    
    print(f'Who said this? {quote}')
    
    for tries in range(5,-1,-1):
        
        answer = input('Guess: ')
        print('\n')
        
        if answer == auth or answer == auth.lower():
            print(f'Congrats You Won! | {tries} attempts remaining')
            break
              
        elif tries == 3:
            print(f'Need a clue? The author was born on {authBornDate} {authBirthPlace} | {tries} attempts remaining')
            
        elif tries == 2:
            print(f'Here’s a clue: Author\'s first name starts with \"{auth[0]}\" | {tries} attempts remaining')
    
        elif tries == 1:
            print(f'Here’s a clue: Author\'s last name starts with \"{lastNameHint(auth)}\" | {tries} attempt remaining')
    
        elif tries == 0:
            print(f'The correct answer was \"{auth}\" Better luck next time!| {tries} attempts remaining')
            
        else:
            print(f'No Problem! Try Again | {tries} attempts remaining')
            continue
            
    #Appending time, game winnig status & remaining attempts to dictionary 
    table['Time'].append(timeNow())
    table['Won-Lost'].append(wonOrlost(tries))
    table['Attempts Remaining'].append(tries)

    print('\n')
    #Asking if player wants to contunue
    wantToPlay = input('Do you want to play again (Y/N):')
    while wantToPlay not in ['n', 'N', 'y', 'Y']:
        print('INVALID RESPONSE')
        wantToPlay = input('Do you want to play again (Y/N):')
    print('\n')

#After termination storing dictionary as df       
print('GAME TERMINATED')
df = pd.DataFrame(table)

Who said this? I am good, but not an angel. I do sin, but I am not the devil. I am just a small girl in a big world trying to find someone to love.


Guess:  jane austen




No Problem! Try Again | 5 attempts remaining


Guess:  j.k. rowling




No Problem! Try Again | 4 attempts remaining


...



Who said this? The trouble with having an open mind, of course, is that people will insist on coming along and trying to put things in it.


Guess:  terry pratchett




Congrats You Won! | 5 attempts remaining




Do you want to play again (Y/N): n




GAME TERMINATED


## Saving Game Scores to Downloads Folder

In [5]:
#Storing the csv file in downloads folder
#If you want to change the location of csv file, change the path variable to desired directory
path = r'Downloads'
os.makedirs(path, exist_ok=True)
csv_file = os.path.join(path, 'Quotes Scrapping Game Data.csv')

try:
    with open(csv_file, 'x') as file:
        df.to_csv(file, index=False)
except FileExistsError:
    df.to_csv(csv_file, mode='a', header=False, index=False)

## Conclusion

In this project, we developed a Quote Guessing Game that scrapes quotes from [quotes.toscrape.com](https://quotes.toscrape.com) using **BeautifulSoup**. Players have 5 attempts to guess the author of a randomly presented quote, with hints provided on the 3rd, 4th & 5th tries. After all attempts, the game asks if the player wants to continue or terminate. Scores are saved locally in a CSV file in the user's **Downloads folder** for future reference. This project demonstrates web scraping, game logic, and data storage.